In [ ]:
from prefect import flow, task
from prefect.task_runners import DaskTaskRunner
from typing import List
import httpx

@task(retries=3)
def get_star_count(repo_url: str) -> int:
    return httpx.get(repo_url).json()["stargazers_count"]

@flow
def get_repo_stars(repo: str):
    url = f"https://api.github.com/repos/{repo}"
    count = get_star_count(url)
    print(f"{repo} has {count} stars!")

@flow(name="Github Stars", task_runner=DaskTaskRunner())
def github_stars(repos: List[str]):
    for repo in repos:
        get_repo_stars(repo)

# run the flow!
if __name__ == "__main__":
    github_stars(["PrefectHQ/Prefect", "PrefectHQ/miter-design"])